## Inlämning 1
Jag tänker ofta på svenska, även i text. Har försökt börja använda engelska för att underlätta användandet av AI verktyg men jag är inte riktigt där än. 
Här är repon: https://github.com/Tuchulcha/Inlamning1
git och jag är inte vänner än, så den kanske inte är updaterad.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load dataset
file_path = "AmazonDataSales.csv"
df = pd.read_csv(file_path)

In [ ]:
#DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False. Interesting
#Using low_memory=False to get more accuracy in inference of datatypes. Increases the odds of pandas choosing the right datatype somehow?
df = pd.read_csv(file_path, low_memory=False)

In [ ]:
# Display the first few rows of the dataframe
df.head()

In [ ]:
# Display basic information about the dataset
num_columns = df.shape[1]
#pd.set_option('display.max_info_columns', num_columns)
df.info(num_columns)

In [ ]:
# Find out more about columns with mixed data types
mixed_type_columns = []
for i, col in enumerate(df.columns):
    types = {type(v) for v in df[col]}
    if len(types) > 1:
        mixed_type_columns.append((i, col))
        print(f"Data types in column {i} {col}:", types)

print(f"{len(mixed_type_columns)} Columns with mixed types")


In [ ]:
# Display statistical overview of numerical columns
df.describe()

In [ ]:
# Calculating and displaying the number of missing values in each column 
df.isnull().sum()

## Data Cleaning
# Handle missing values and data inconsistencies
Replace missing values in 'Courier Status', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-postal-code', and 'ship-country' with appropriate values or methods

In [ ]:
# Fill in missing 'Courier Status' with "Unknown"
# df['Courier Status'].fillna('Unknown', inplace=True) Seems like inplace=True is being disbanded
df['Courier Status'] = df['Courier Status'].fillna('Unknown')

In [ ]:
# Previous row fill missing 'currency'
df['currency'] = df['currency'].ffill(axis = 0)
print(len(df[df['currency'].apply(lambda x: isinstance(x, float))]))

In [ ]:
# Fill missing 'Amount' with mean
df['Amount'] = df['Amount'].fillna(value=df['Amount'].mean() )

In [ ]:
#Explore 'ship-city', 'ship-state', 'ship-postal-code', and 'ship-country'
#Maybe city and state isn't missing when country is, for example.
# Filter the DataFrame to only include rows with missing values in the specified columns
missing_values_df = df[df[['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']].isna().any(axis=1)]

missing_values_df

The values are misisng for all four columns at the same time, so no possibility of infering country from state and city

In [ ]:
#Fill in missing values in 'ship-city', 'ship-state', 'ship-postal-code', and 'ship-country' with "Missing"
df[['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']] = df[['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']].fillna('Missing')

In [ ]:
# Drop columns with a high percentage of missing values if they are not crucial
df.drop(columns=['fulfilled-by', 'Unnamed: 22', 'promotion-ids'], inplace=True)

# Data Cleaning
## Check for and resolve any data inconsistencies, such as incorrect data types or unusual values


In [ ]:
# Datatypes look ok
df.dtypes

In [ ]:
# No negative values in 'Qty' or 'Amount'
df.describe()

In [ ]:
# Check for columns with mixed data types again
mixed_type_columns = []
for i, col in enumerate(df.columns):
    types = {type(v) for v in df[col]}
    if len(types) > 1:
        mixed_type_columns.append((i, col))
        print(f"Data types in column {i} {col}:", types)

print(f"{len(mixed_type_columns)} Columns with mixed types")


currency? First time I ran this code it was only 'ship-postal-code' that was mixed type. Not this column. Maybe I just missed it but I will go back and check.
I had missed df['currency'] = or inplace= operator
before df['currency'].ffill(axis = 0)
It's fixed now but I'll leave this in to show that it was a process. Also inplace= got "decrapiated" while I was doing this, maybe my laptop which i proof read on got a newer version of pandas installed? 
The below code was to explore if something was off with 'currency' but I think I just missed the inplace operator or df['currency'] = before df['currency'].ffill(axis = 0)
As mentioned.

In [ ]:
# Exploring 'currency'
# Filter the DataFrame to include only rows where the value in 'currency' is a float
float_values_df = df[df['currency'].apply(lambda x: isinstance(x, float))]

# Count and display the occurrences of each unique float value
float_value_counts = float_values_df['currency'].value_counts()
print(float_value_counts)

print(len(df[df['currency'].apply(lambda x: isinstance(x, float))]))

# Count NaN values in the column
nan_count = df['currency'].isna().sum()
print(f"Number of NaN values in 'currency': {nan_count}")

# Count non-NaN float values in the column
non_nan_floats = df['currency'].apply(lambda x: isinstance(x, float) and not pd.isna(x)).sum()
print(f"Number of non-NaN float values in 'currency': {non_nan_floats}")



In [ ]:
# Looking at df.ddtypes it's clear that all object type columns should be string datatype except 'Date'. 
# The float types in the Data types in column 18 'ship-postal-code' will be converted to string when the dtype of the column is changed from object to string.

# Convert all object type columns to string type
df = df.astype({col: 'string' for col in df.select_dtypes('object').columns})


#### I wonder what compute cost needlessly converting the 'Date' column carries.
I had this thought that it takes unecessary compute to convert the 'Date' column to a string type when it will be converted later to date type. This is correct. But how I went about trying it was wrong. Of course the right comparison is not between converting all columns with one line of code and making a loop checking if every column is the 'Date' column and converting only if it isn't.
The right comparison would be to convert all the columns with one command vs making an explicit list with all the columns to be converted (this list containing all columns except the 'Date' column). The caveat being that this is not 'future' proof. Maybe a more salient comparison is making a list using code, and removing 'Date' from this list with code, to account for if the number of columns changes? Hmmm. Maybe it's not easy to say what the right comparison is...

import time


start_time = time.time()
df = df.astype({col: 'string' for col in df.select_dtypes('object').columns})
end_time = time.time()
time_with_date_conversion = end_time - start_time


start_time = time.time()
exclude_columns = ['Date']
df = df.astype({col: 'string' for col in df.select_dtypes('object').columns if col not in exclude_columns})
end_time = time.time()
time_without_date_conversion = end_time - start_time


print(f"Time with date conversion: {time_with_date_conversion} seconds")
print(f"Time without date conversion: {time_without_date_conversion} seconds")


### Date problems
Okay, something is not right with the 'Date' data. Manually looking at the data i find dates such as "2004-01-22" and "05-31-22". The only way the latter date works is if it's in the format "MM-DD-YY". Looking at the data Index 72469 to 72470 goes from 05-13-22 to 2005-12-22. Index 90997 to 90998 goes from 2005-01-22 to 04-30-22. It seems like there's a mix of formats and I find the jump in years from 2022 to 2005 and then from 2005 to 2022 to be odd.

Since my manual sampling shows dates that can only be YYYY-MM-DD and dates that can only be MM-DD-YY. If I change YYYY-MM-DD to MM-DD-YY the dates will be congruent given that my observation holds for the data. One way to do this could be to check the length of the value in each row in the column.

pandas seems to support more advanced checks like checking the current format of the date per row and doing some kind of conversion

So a single line of code was enough to do the conversion: df['Date'] = pd.to_datetime(df['Date'], format='%m-%d-%y', errors='coerce') 
Leaving this block in here as a memento of the hours spent prompting trying to find some other solution




In [ ]:

df['Date'] = pd.to_datetime(df['Date'], format='%m-%d-%y', errors='coerce')

# Count the number of NaT values in the 'Date' column
nat_count = df['Date'].isna().sum()
print(f"Number of NaT values in 'Date' column: {nat_count}")


In [ ]:
# Check datatypes again
df.dtypes

### Standardize text data for uniformity (e.g., Category, Size, Style).

In [ ]:
# Convert all text to lowercase for uniformity
df['Category'] = df['Category'].str.lower()
# Or capitalize the first letter of each word
#df['Category'] = df['Category'].str.title()
# is it for machine learning that all lower is better? 

In [ ]:
# Add one for 'Size' and 'Style' as well
print(f"Size strings that were entirely uppercase: {len(df['Size'].str.isupper())} out of {len(df['Size'])}")
print(f"Style strings that were entirely uppercase: {len(df['Style'].str.isupper())} out of {len(df['Style'])}")
# Same here, is it for machine learning that all lower is better? It's already consistent and it looks
# better the way that it is now. But sure!
df['Size'] = df['Size'].str.lower()
df['Style'] = df['Style'].str.lower()

In [ ]:
# Lets assume that all string values in all columns are supposed to be lower case
for col in df.select_dtypes(include=['string']):
    df[col] = df[col].str.lower()

In [ ]:
# Remove leading and trailing spaces

#First check if there are any leading or trailing spaces in string acolumns
columns_with_trailspace = 0
columns_with_leadspace = 0
for col in df.select_dtypes(include=['string']):
    trailing_spaces = df[col].str.endswith(' ')
    leading_spaces = df[col].str.startswith(' ')
    if trailing_spaces.any():
        print(f"Column '{col}' has trailing spaces.")
        columns_with_trailspace += 1
    if leading_spaces.any():
        print(f"Column '{col}' has leading spaces.")
        columns_with_leadspace += 1
print(f"amount of columns with rows that have trailing spaces: {columns_with_trailspace}")
print(f"amount of columns with rows that have leading spaces: {columns_with_leadspace}")


In [ ]:
# Remove leading and trailing spaces
df['ship-state'] = df['ship-state'].str.strip()

Jag kanske går ån efter vatten här? Är det värt att kolla efter whitespace att trimma? Det kanske är mer compute effektivt att bara trimma direkt, utan att veta om det finns något att trimma?

Att kolla efter space att trimma går igenom all data en gång, att trimma allt går igenom data en gång. Så... kanske bara bättre att trimma direkt?

In [ ]:
# Remove leading and trailing whitespace from all column names
df.columns = df.columns.str.strip()

### Replace any inconsistencies or typos, if known
What columns could be candidates?
Status, Fulfilment, Sales Channel, ship-service-level, Category, Courier Status, ship-city,ship-state


In [ ]:
# List of columns to apply value_counts
columns_to_check = ['Status', 'Fulfilment', 'Sales Channel', 
                    'ship-service-level', 'Category', 'Size', 'Courier Status', 
                    'ship-city', 'ship-state']

# Loop through each column and print the value counts
for col in columns_to_check:
    try:
        print(f"Value counts for '{col}':")
        print(df[col].value_counts())
    except KeyError:
        print(f"Column '{col}' not found in DataFrame.")
    print("\n")


In [ ]:
# Check 'Sales Channel' column separately because it didn't work
try:
    print("Value counts for 'Sales Channel':")
    print(df['Sales Channel'].value_counts())
except KeyError as e:
    print(f"An error occurred: {e}")
# Don't know why it doesn't work, but it looks right manually checking it.
# Ah! figured it out, whitespace in the column name! I will correct it higher up in the code 
# With the other leading and trailing whitespace removal. Leaving this for posterity

In [ ]:
#'ship-city' and 'ship-state' has obvious typos. Ship-city a lot more than ship-state.

tail_value_counts = df['ship-city'].value_counts().tail(15)
print(tail_value_counts)

tail_value_counts = df['ship-state'].value_counts().tail(15)
print(tail_value_counts)


In [ ]:
# How many are there?
sample_ship_cities = df['ship-city'].unique()
print(len(sample_ship_cities))  
#Yeah... that's too much for my scope

In [ ]:
sample_ship_state = df['ship-state'].unique()
print(len(sample_ship_state))
#This is manageable
sample_ship_state[:100]  # Displaying the first 100 unique state names as a sample

In [ ]:
# Lets try to fix some of them. Let's check the tail for good candidates
tail_value_counts = df['ship-state'].value_counts().tail(48)
print(tail_value_counts)

In [ ]:
# rajsthan, rj, rajshthan Corrects to: rajasthan
# pondicherry Corrects to: puducherry
# orissa Corrects to: odisha
# I'm not sure what the corrections are for the rest so let's settle for these.
# Create a mapping dictionary for the corrections
corrections = {
    'rajsthan': 'rajasthan',
    'rj': 'rajasthan',
    'rajshthan': 'rajasthan',
    'pondicherry': 'puducherry',
    'orissa': 'odisha'
}

# Apply the corrections to the 'ship-state' column
df['ship-state'] = df['ship-state'].replace(corrections)

In [ ]:
# Lets assume that it's a good thing to standardize column headers as well!
# Let's go with lowercase all the way.
df.columns = df.columns.str.lower()


In [ ]:
# Recheck the dataset after standardizing
df.head()

In [ ]:
# Recheck the dataset after standardizing
df.describe()

In [ ]:
# Recheck the dataset after standardizing
df.dtypes

To summarize, we've done some exploration, looked for inconsistencies and missing values, filled in missing values. Specified data-type for columns, made all string values lower-case, as well as the column headers. Fixed some spelling error.
Left in some of the process instead of making it stream-lined for austerity since this is for my education, not production.

I think this ends the DataProcessMLSemi.py part. According to the pdf Data_PreProcessing_Exercise this is left:
3- Feature engineering
4- Data Transformation
5- Presentation and Documentation

### Feature Engineering

In [ ]:
# Make month and year columns
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [ ]:
# Make a b2b binary column
df['b2b_binary'] = df['b2b'].astype(int)

In [ ]:
# Group 'amount' by interval
# This makes four bins with the same number of rows being labeled with each
# Use pd.qcut() to create the 'Label' column based on the distribution of 'Value'
df['amount_quartiles_bin'] = pd.qcut(df['amount'], q=4, labels=['0-25', '25-50', '50-75', '75-100'])

In [ ]:
# Making sure there aren't a boatload of 0 values
#print(df.nsmallest(20, 'amount'))
min_value = df['amount'].min()
print((df['amount'] == min_value).sum())
# This is fine, maybe, I guesstimate.

In [ ]:
# Making sure the max value isn't a complete outlier
n = 100  # For example, the top 10 largest values

# Display the n largest values in the 'Amount' column
largest_amounts = df['amount'].nlargest(n)
print(largest_amounts)
# This is fine, maybe, I guesstimate.
# Say that it wasn't fine and the outliers skews the data df['amount_Log'] = np.log1p(df['amount']) would be a way to fix it. You could say the difference between the size of 
# numbers gets smaller and smaller as the size of the difference increases if you "log" it
# or you could just "do one dirty" and chop off the top 3 values from the generation of the normalized column

In [ ]:
# I'm interested in making labels where the labels are min to max divided in to four labels
# max for 'amount' is 5584. So 0-25, 25-50, 50-75, 75-100 is:
# 0-1396, 1397-2792, 2793-4188, 4189-5584

# Define custom bin edges
custom_bins = [0, 1396, 2792, 4188, 5584]

# Define labels for the bins
labels = ['0-25', '25-50', '50-75', '75-100']

# Use pd.cut() to create the 'Label' column based on custom bins
df['amount_relative_range_bins'] = pd.cut(df['amount'], bins=custom_bins, labels=labels, include_lowest=True)

In [ ]:
# Normalize 'qty' and 'amount'
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max Normalization to 'qty' and 'amount'
df[['qty_minmax_norm', 'amount_minmax_norm']] = scaler.fit_transform(df[['qty', 'amount']])


Note: somehow I messed up using git and github. I was mostly done, just doing some polish but was going away for the weekend so I made a repo for working on my laptop. My guess is that I didn't save in vscode on my desktop before committing and pushing. Then I pulled the file on my laptop, did some changes without scrolling to the end of the file, making sure it was correct. Committed and pushed, came home, pulled and in the process lost what I had done before I went away. But I don't know. git and github makes me confused. Anyway. Lost the work I'd done on one-hot encoding and describing what I had done. I still have the csv file from what I did. It's called "AmazonCleanedAndSpiced.csv" and is in my repo. I guess "I messed up saving my file and pushing" is my version of "the dog ate my homework"
I'll power through and remake it with a bit less care as it's sunday, late, and I'm pooped.

### Thoughts about categorical columns
Some columns that could be considered categorical have high cardinality so I'll skip them because of size.
sku and asin are both high cardinality and probably identifiers

In [ ]:

# Encode categorical columns with one-hot encoding
categorical_columns = ['Status', 
                       'Fulfilment', 
                       'Sales Channel', 
                       'ship-service-level', 
                       'Category', 
                       'Courier Status' 
                       #'ship-city', 
                       #'ship-state', 
                       #'ship-country', 
                       #'Style'
                       ]

# Apply one-hot encoding with a prefix
for col in categorical_columns:
    dummies = pd.get_dummies(df[col], prefix=f"o-he_{col}")
    df = pd.concat([df, dummies], axis=1)

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('AmazonCleanedAndQuickSpiced.csv', index=False)


With this the data has been looked over, missing values filled in, some spelling errors have been sorted, a few derivated columns have been created. Normalized columns have been created as well as one-hot encoded for a few categorical columns. Data and columns names have been changed to all lower-case..